<a href="https://colab.research.google.com/github/caseykotowski/Song-Features-Over-Time/blob/DB_Segment2_MCC/MCC_Loading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingData").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Load data from S3 Buckets into Spark DataFrame

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/tracks.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("tracks.csv"), sep=",", header=True)

In [4]:
# Show DataFrame
df.show()

+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|                  id|                name|popularity|duration_ms|explicit|            artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|
+--------------------+--------------------+----------+-----------+--------+-------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+
|35iwgR4jXetI318WE...|               Carve|         6|     126903|       0|            ['Uli']|['45tIt06XoI0Iio4...|  1922-02-22|       0.645| 0.445|  0| -13.338|   1|      0.451|       0.674|           0.744|   0.151|  0.127|104.851|             3|


In [5]:
# Print our schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- id_artists: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)



In [6]:
# Read in artist data from S3 Buckets
from pyspark import SparkFiles
url = "https://spot-i-fy-bucket.s3.us-east-2.amazonaws.com/artists.csv"
spark.sparkContext.addFile(url)
a_df = spark.read.csv(SparkFiles.get("artists.csv"), sep=",", header=True)

# Show DataFrame
a_df.show()

+--------------------+---------+------+--------------------+----------+
|                  id|followers|genres|                name|popularity|
+--------------------+---------+------+--------------------+----------+
|0DheY5irMjBUeLybb...|      0.0|    []|Armid & Amir Zare...|         0|
|0DlhY15l3wsrnlfGi...|      5.0|    []|         ปูนา ภาวิณี|         0|
|0DmRESX2JknGPQyO1...|      0.0|    []|               Sadaa|         0|
|0DmhnbHjm1qw6NCYP...|      0.0|    []|           Tra'gruda|         0|
|0Dn11fWM7vHQ3rinv...|      2.0|    []|Ioannis Panoutsop...|         0|
|0DotfDlYMGqkbzfBh...|      7.0|    []|       Astral Affect|         0|
|0DqP3bOCiC48L8SM9...|      1.0|    []|           Yung Seed|         0|
|0Drs3maQb99iRglyT...|      0.0|    []|               Wi'Ma|         0|
|0DsPeAi1gxPPnYjgp...|      0.0|    []|             lentboy|         0|
|0DtvnTxgZ9K5YaPS5...|     20.0|    []|            addworks|         0|
|0Du4t8UK1lBD8TnOI...|      1.0|    []|         Harrbazzter|    

In [7]:
a_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: string (nullable = true)



# Create Pandas DataFrames

In [8]:
import pandas as pd
import numpy as np
import datetime

In [9]:
tracks_df = df.toPandas()
tracks_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.263,0,-22.136,1,0.957,0.797,0.0,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.177,1,-21.18,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918,0.104,0.397,169.98,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.158,3,-16.9,0,0.039,0.989,0.13,0.311,0.196,103.22,4


In [10]:
artists_df = a_df.toPandas()
artists_df.head()

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [11]:
artists_df = artists_df.drop(columns=['followers'])
artists_df.head(10)

,id,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,[],Ioannis Panoutsopoulos,0
5,0DotfDlYMGqkbzfBhcA5r6,[],Astral Affect,0
6,0DqP3bOCiC48L8SM9gK4W8,[],Yung Seed,0
7,0Drs3maQb99iRglyTuxizI,[],Wi'Ma,0
8,0DsPeAi1gxPPnYjgpiEGSR,[],lentboy,0
9,0DtvnTxgZ9K5YaPS5jdlQW,[],addworks,0


# Clean up Data

In [12]:
print(tracks_df.count())
tracks_df = tracks_df.dropna()
print(tracks_df.count())
tracks_df = tracks_df.drop_duplicates()
print(tracks_df.count())

id                  586672
name                586601
popularity          586672
duration_ms         586672
explicit            586672
artists             586672
id_artists          586660
release_date        586660
danceability        586660
energy              586660
key                 586660
loudness            586660
mode                586660
speechiness         586660
acousticness        586660
instrumentalness    586660
liveness            586660
valence             586660
tempo               586660
time_signature      586660
dtype: int64
id                  586589
name                586589
popularity          586589
duration_ms         586589
explicit            586589
artists             586589
id_artists          586589
release_date        586589
danceability        586589
energy              586589
key                 586589
loudness            586589
mode                586589
speechiness         586589
acousticness        586589
instrumentalness    586589
liveness       

In [14]:
 # tracks_df['release_year'] = tracks_df['release_date'].astype(str).str.extract('(\d{4})').astype(int)

#tracks_df['release_year'] = pd.to_datetime(tracks_df['release_date'], errors='coerce').dt.year
# print(tracks_df.count())

Drop duplicates and incomplete rows

In [15]:
print(artists_df.count())
artists_df = artists_df.dropna()
print(artists_df.count())
artists_df = artists_df.drop_duplicates()
print(artists_df.count())

id            1104349
genres        1104349
name          1104349
popularity    1104349
dtype: int64
id            1104349
genres        1104349
name          1104349
popularity    1104349
dtype: int64
id            1104349
genres        1104349
name          1104349
popularity    1104349
dtype: int64


# Write DataFrame to RDS

In [8]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-15 03:22:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-08-15 03:22:14 (6.62 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [9]:
 # Configuration for RDS instance
mode = "append"
jdbc_url="jdbc:postgresql://spotify-db.c3uki6gnymem.us-east-2.rds.amazonaws.com:5432/spotify"
config = {"user":"root", 
          "password": "AWS2021!", 
          "driver":"org.postgresql.Driver"}

In [11]:
df.write.jdbc(url=jdbc_url, table="tracks_df", mode="append", properties=config)

In [12]:
a_df.write.jdbc(url=jdbc_url, table="artist_df", mode="append", properties=config)